# MoveNet Python

## Imports

In [3]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

## Load Model

In [ ]:
interpreter = tf.lite.Interpreter(model_path='movenet-tflite-singlepose-lightning-v1.tflite')
interpreter.allocate_tensors()

## Detections

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [ ]:
# Define keypoint indices (MoveNet Lightning)
NOSE = 0
LEFT_SHOULDER = 5
RIGHT_SHOULDER = 6
LEFT_HIP = 11
RIGHT_HIP = 12

# Simple helper: compute angle between 3 points (in degrees)
def angle_between(p1, p2, p3):
    """
    Returns the angle at p2 formed by p1-p2-p3
    """
    a = np.array(p1) - np.array(p2)
    b = np.array(p3) - np.array(p2)
    cosine_angle = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return degrees(angle)

# Compute slope for forward/backward leaning
def lean_angle(shoulder, hip):
    dx = shoulder[0] - hip[0]
    dy = shoulder[1] - hip[1]
    return degrees(atan2(dy, dx))

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess for MoveNet
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    # MoveNet inference
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])[0, 0, :, :]

    # Scale keypoints to original frame size
    h, w, _ = frame.shape
    keypoints = []
    for kp in keypoints_with_scores:
        y, x, score = kp
        keypoints.append((int(x * w), int(y * h), score))

    # Draw MoveNet keypoints
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    # ----- Posture Calculations -----
    # Check confidence before using keypoints
    min_conf = 0.3
    left_shoulder = keypoints[LEFT_SHOULDER] if keypoints[LEFT_SHOULDER][2] > min_conf else None
    right_shoulder = keypoints[RIGHT_SHOULDER] if keypoints[RIGHT_SHOULDER][2] > min_conf else None
    left_hip = keypoints[LEFT_HIP] if keypoints[LEFT_HIP][2] > min_conf else None
    right_hip = keypoints[RIGHT_HIP] if keypoints[RIGHT_HIP][2] > min_conf else None
    nose = keypoints[NOSE] if keypoints[NOSE][2] > min_conf else None

    # Approximate back angle (torso lean)
    if left_shoulder and right_shoulder and left_hip and right_hip:
        mid_shoulder = ((left_shoulder[0] + right_shoulder[0])//2,
                        (left_shoulder[1] + right_shoulder[1])//2)
        mid_hip = ((left_hip[0] + right_hip[0])//2,
                   (left_hip[1] + right_hip[1])//2)
        lean = lean_angle(mid_shoulder, mid_hip)
        cv2.putText(frame, f'Torso lean: {lean:.1f} deg', (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    # Approximate neck angle
    if nose and left_shoulder and right_shoulder:
        mid_shoulder = ((left_shoulder[0] + right_shoulder[0])//2,
                        (left_shoulder[1] + right_shoulder[1])//2)
        neck_angle = angle_between(nose, mid_shoulder, mid_hip)
        cv2.putText(frame, f'Neck angle: {neck_angle:.1f} deg', (30, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    # Visualize key posture points
    for pt in [mid_shoulder, mid_hip, nose]:
        cv2.circle(frame, pt, 5, (255,0,0), -1)

    cv2.imshow('MoveNet Posture', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()